In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests

from repeater import retry

NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

Wallet Import

In [6]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# NID = 1

# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# NID = 2

# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
NID = 3

# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# NID = 80

# Custom
# icon_service = IconService(HTTPProvider("http://127.0.0.1:9000/", 3))

### Load wallet using Keystore file

In [7]:
wallet = KeyWallet.load("location_to_keystore","PASSWORD")
deployer_wallet = wallet.get_address()
deployer_wallet

Raised KeyStoreException while loading the wallet by the keystore file because the file is not found.
Traceback (most recent call last):
  File "/home/swopnil/anaconda3/envs/venv2/lib/python3.7/site-packages/iconsdk/wallet/wallet.py", line 101, in load
    with open(file_path, 'rb') as file:
FileNotFoundError: [Errno 2] No such file or directory: 'location_to_keystore'


KeyStoreException: File is not found. (Key store error)

### Load wallet using Wallet Private Key

In [10]:
private="ca86f377ea9d2420104058b405bbd61d4214f0fff4da0e4a33d6316ca6224024"
wallet = KeyWallet.load(bytes.fromhex(private))
prep_address = wallet.get_address()
prep_address

'hxff54214b12f771f0515afe4c0a7d3742a74fb380'

# Deploying Contract

In [12]:
params = {}

deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(NONCE)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content("my_first_score"))\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xf22fe6eafd5a3cc37de090e18023e512334819925cb7182fc47b07dafb8be131',
 'blockHeight': 13206454,
 'blockHash': '0x5cf171ed6445dfa28a7320c032e544fbba861db24e045b8b136a3c3872ea6396',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx06ea50a7eb3739067cc02cc40813f1fecc67bae5',
 'stepUsed': 1245468000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1245468000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
contract_address = "cx0000...000"

### Contract Update

In [ ]:
# Generates an instance of transaction for deploying score

transaction = DeployTransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(contract_address) \
            .nid(NID) \
            .nonce(100) \
            .content_type("application/zip") \
            .content(gen_deploy_data_content("SCORE.ZIP")) \
            .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, wallet,step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)